In [28]:
import joblib
EXP_PATH_NAME="SCNN-OCSVM"
RIVAL_PATH_NAME="WACA-OCSVM"
joblib.cpu_count()

32

In [29]:
# !pip install --upgrade pip

import os
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb

WINDOW_SIZE_LST=[125, 250, 500, 750, 1000]
np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neural Networks utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER

In [30]:
import sklearn
sklearn.__dir__()
sklearn.__version__
WINDOW_SIZE_LST

[125, 250, 500, 750, 1000]

In [31]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [32]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="OCSVM"
P.ocsvm_kernel = "rbf"
P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function


# test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{RIVAL_PATH_NAME}-df.json"
# RIVAL_DF = pd.read_json(test_file_name)
# RIVAL_DF
# # Convert the string representation of tuples back to tuples
# idx_tuples = [eval(i) for i in RIVAL_DF.columns.tolist()]

# # # Convert list of tuples back to a MultiIndex
# multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

# RIVAL_DF.columns = multiindex_columns
# RIVAL_DF

In [33]:
P.smoothing = "Butterworth"


#range(1000, 2001, 250)#
preprocessing_method="Naive"
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
rival_test_hyperparameters_df


,window_size,step_width,Mean_EER,median_nu,median_gamma,Gini_coef,cut_off_freq
0,125,62,0.226060,0.035779,0.655129,0.355660,41
1,250,125,0.173420,0.055147,0.372759,0.431020,41
2,500,250,0.144618,0.041895,0.372759,0.499510,41
3,750,375,0.142465,0.034759,0.120679,0.503772,41
4,1000,500,0.125610,0.035779,0.068665,0.566719,41
5,1250,625,0.126424,0.040876,0.068665,0.587064,41
6,1500,750,0.120782,0.033740,0.068665,0.588185,41
7,1750,875,0.104300,0.031701,0.068665,0.656070,41
8,2000,1000,0.095222,0.034759,0.068665,0.669817,41


In [34]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [35]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [36]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [37]:
randomized_data_idx = list(range(len(P.user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
split_idx = 2 * (len(randomized_data_idx)//3) + 1
train_set = randomized_data_idx[: split_idx]
test_set = randomized_data_idx[split_idx: ]
# train_set = randomized_data_idx
print(f"train_set: {train_set}\ntest_set: {test_set}")
# train_set = test_set
# test_set = train_set
print(f"train_set: {train_set}\ntest_set: {test_set}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [38]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [39]:
print(f"train_set: {train_set}")
# print(f"X_exp1_train_dic: {X_exp1_train_dic.keys()}")
# print(f"X_exp2_train_dic: {X_exp2_train_dic.keys()}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]


In [40]:
print(f"test_set: {test_set}")
# print(f"X_exp1_test_dic: {X_exp1_test_dic.keys()}")
# print(f"X_exp2_test_dic: {X_exp2_test_dic.keys()}")

test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [41]:
WINDOW_SIZE_LST

[125, 250, 500, 750, 1000]

In [42]:
extract_learnt_features

<function __main__.extract_learnt_features(X_exp1_dic, X_exp2_dic, fitted_scaler_classifier_exp2_dic, deep_feature_model, verbose=1)>

In [43]:
def get_exp_deep_feature_extractors_dict(exp_num, model_architectures= list(TRAINING_CONFIG_DICT['1000'].keys())):
    exp_deep_feature_extractors_dict={}

    model_classifier_type=EXP_PATH_NAME
    base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{model_classifier_type}/{exp_num}"
    for archi_name in model_architectures:
        # archi_name="SCNN_3_1_conv_1_dense_arg_dict_default"
        path_to_archi_name=base_path+f"/{archi_name}"
        archi_win_dict={}
        for winsize in WINDOW_SIZE_LST:
            final_path=path_to_archi_name+f"/{winsize}"
            deep_feature_extractor_path=final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-deep_feature_extractor"
            print(deep_feature_extractor_path)
            
            deep_feature_model=load_model(deep_feature_extractor_path)#, custom_objects={'loss_func': loss_func})
            
            deep_feature_extractor = partial_func(extract_learnt_features, deep_feature_model=deep_feature_model)

            archi_win_dict[winsize]=deep_feature_extractor

        exp_deep_feature_extractors_dict[archi_name]=archi_win_dict
        
    return exp_deep_feature_extractors_dict

# exp_deep_feature_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_1)

In [44]:
# Create a function to suppress TensorFlow warnings
def suppress_tf_warnings():
    warnings.filterwarnings('ignore', category=UserWarning)
    tf.get_logger().setLevel('ERROR')

# Call the function before loading the model
suppress_tf_warnings()

def enable_tf_warnings():
    warnings.filterwarnings('default', category=UserWarning)
    tf.get_logger().setLevel('WARN')

# Call the function to enable warnings
# enable_tf_warnings()

In [45]:
TESTING_CONFIG_DICT.items()

NameError: name 'TESTING_CONFIG_DICT' is not defined

In [ ]:

# X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_SNN_train_exp2_dict=get_raw_windows(raw_dfList_exp1, raw_dfList_exp2, 1750, 1750//2, train_set, 
#                                                                     P.scaler, P.num_sample_points_per_exp, EMA_per_win_span=None, SMA_per_win_winsize=None)


In [ ]:
# X_train_exp1_dict[7][0].shape

In [28]:
# TESTING_CONFIG_DICT["SCNN_1_3_conv_1_dense_arg_dict_default"]

In [29]:
# a=np.array([X_train_exp1_dict[7][0]])

In [30]:
# for model_name, model in TESTING_CONFIG_DICT.items():
#     print(model[1750].predict(a))

In [31]:
# archi_name="SCNN_1_2_conv_1_dense_arg_dict_default"
# base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{model_classifier_type}/{EXP_NUM_1_1}"
# path_to_archi_name=base_path+f"/{archi_name}"
# final_path=path_to_archi_name+f"/{winsize_str}"
# deep_feature_extractor_path=final_path+f"/{model_classifier_type}-{EXP_NUM_1_1}-{archi_name}-{winsize_str}-deep_feature_extractor"
# model=load_model(deep_feature_extractor_path)

In [32]:
# model.summary()

In [33]:
# base_path=f"final_experiments_results/{SAVE_PATH_NAME}/{EXP_PATH_NAME}/{EXP_NUM_1_1}"
# path_to_archi_name=base_path+f"/{archi_name}"
# final_path=path_to_archi_name+f"/{winsize}"
    
#     if not os.path.isdir(base_path):
#         raise Exception(f"Base path does not exist: {base_path}")
        
#     if not os.path.isdir(path_to_archi_name):
#         os.mkdir(path_to_archi_name)

#     if not os.path.isdir(final_path):
#         os.mkdir(final_path)
        
#     model.save(final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-model")
#     deep_feature_model = extract_deep_feature_extactor(model)
#     deep_feature_model.save(final_path+f"/{model_classifier_type}-{exp_num}-{archi_name}-{winsize}-deep_feature_extractor")

In [34]:

# def correct_archi_extractor_dict_key_names(exp_deep_archi_extractors_dict):
#     items=list(exp_deep_archi_extractors_dict.items())
#     for key, val in items:
#         exp_deep_archi_extractors_dict[key[:-len("_arg_dict_default")]]=val
#         # del exp_deep_archi_extractors_dict[key]
#     return exp_deep_archi_extractors_dict
# exp_deep_archi_extractors_dict=correct_archi_extractor_dict_key_names(exp_deep_feature_extractors_dict)
# exp_deep_archi_extractors_dict.keys()

In [35]:
# exp_deep_archi_extractors_dict.keys()
# WINDOW_SIZE_LST = [125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
# WINDOW_SIZE_LST=WINDOW_SIZE_LST[:-2]

In [18]:
enable_tf_warnings()


In [ ]:
RIVAL_PATH_NAME

In [34]:
# rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
# rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

# P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
# print(f"cut_off_freq: {P.cut_off_freq}")
# rival_test_hyperparameters_df

In [35]:
# get_exp_deep_feature_extractors_dict(EXP_NUM_0, model_architectures= [list(TRAINING_CONFIG_DICT['2000'].keys())[0]])


# 0. No Smoothing
### Optimizing and Testing

In [38]:
P.smoothing = None

preprocessing_method=None
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
rival_test_hyperparameters_df

,window_size,step_width,Mean_EER,median_nu,median_gamma,Gini_coef,cut_off_freq
0,125,62,0.231879,0.042914,0.655129,0.343750,-99999
1,250,125,0.178423,0.040876,0.655129,0.429536,-99999
2,500,250,0.152316,0.040876,0.372759,0.494137,-99999
3,750,375,0.146990,0.035779,0.120679,0.515640,-99999
4,1000,500,0.133523,0.034759,0.120679,0.592362,-99999
5,1250,625,0.128229,0.034759,0.068665,0.599732,-99999
6,1500,750,0.137243,0.034759,0.030650,0.571571,-99999
7,1750,875,0.108889,0.035779,0.068665,0.651257,-99999
8,2000,1000,0.106611,0.035779,0.039069,0.648584,-99999


In [39]:
init_experiment_params(exp_config=P)
P.cut_off_freq

reseting experiment params successful!


In [45]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_0)
# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_0, model_architectures= [list(TRAINING_CONFIG_DICT['1500'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = None

preprocessing_method=None
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")


EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    # P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    # print(f"cut_off_freq: {P.cut_off_freq}")

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST,
                                                                                             test_set=test_set, exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                             overlap=OVERLAP, 
                                                                                             best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"test_dict_key: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["scnn_archi_name"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-0-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCS

  0%|          | 0/7 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.3439715448766947
MakeSCNNXExpDicUnknown Time:  35.83786430535838
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8274.42it/s]

 20%|██        | 1/5 [00:00<00:02,  1.81it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 17015.43it/s]

 40%|████      | 2/5 [00:01<00:01,  1.83it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8908.89it/s]

 60%|██████    | 3/5 [00:01<00:01,  1.87it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10785.04it/s]

 80%|████████  | 4/5 [00:02<00:00,  1.76it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8674.88it/s]

 14%|█▍        | 1/7 [00:41<04:06, 41.02s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.0465429280884564
MakeSCNNXExpDicUnknown Time:  25.74198350775987
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 15984.39it/s]

 20%|██        | 1/5 [00:00<00:00,  4.78it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8507.72it/s]

 40%|████      | 2/5 [00:00<00:00,  4.72it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9936.75it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.66it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 17119.61it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.51it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9047.25it/s]

 71%|███████▏  | 5/7 [01:54<00:36, 18.20s/it]

window_size: 1250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9541736948303878
MakeSCNNXExpDicUnknown Time:  8.607104716356844
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9273.28it/s]

 20%|██        | 1/5 [00:00<00:00,  4.69it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9408.49it/s]

 40%|████      | 2/5 [00:00<00:00,  4.90it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5587.93it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.91it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 12862.02it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.02it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8238.66it/s]

 86%|████████▌ | 6/7 [02:05<00:15, 15.67s/it]

window_size: 1500
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9034201209433377
MakeSCNNXExpDicUnknown Time:  8.479093419853598
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9222.30it/s]

 20%|██        | 1/5 [00:00<00:00,  5.29it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9383.23it/s]

 40%|████      | 2/5 [00:00<00:00,  5.02it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 11775.14it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.99it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7826.65it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.97it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6550.53it/s]

100%|██████████| 7/7 [02:16<00:00, 19.45s/it]


In [46]:
# exp_deep_archi_extractors_dict.items()

# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [50]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_1)
# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_1_1, model_architectures= [list(TRAINING_CONFIG_DICT['1000'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    




EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }



    test_dict_key=scnn_archi_name, P.cut_off_freq
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, 
                                                                                             test_set=test_set, exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"cut_off_freq: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["scnn_archi_name", "cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/7 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.3365480490028858
MakeSCNNXExpDicUnknown Time:  36.44707042677328
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 5548.76it/s]

 20%|██        | 1/5 [00:00<00:02,  1.79it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 12350.72it/s]

 40%|████      | 2/5 [00:01<00:01,  1.83it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5855.51it/s]

 60%|██████    | 3/5 [00:01<00:01,  1.83it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8039.69it/s]

 80%|████████  | 4/5 [00:02<00:00,  1.76it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6125.75it/s]

 14%|█▍        | 1/7 [00:41<04:09, 41.60s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.021809108555317
MakeSCNNXExpDicUnknown Time:  21.222565955948085
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10729.86it/s]

 20%|██        | 1/5 [00:00<00:00,  4.82it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9838.86it/s]

 40%|████      | 2/5 [00:00<00:00,  4.91it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8651.62it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.70it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8176.03it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.84it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6432.98it/s]

 71%|███████▏  | 5/7 [01:53<00:35, 17.94s/it]

window_size: 1250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.8087746039964259
MakeSCNNXExpDicUnknown Time:  8.592254498973489
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10353.75it/s]

 20%|██        | 1/5 [00:00<00:00,  4.86it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6107.02it/s]

 40%|████      | 2/5 [00:00<00:00,  5.00it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8417.23it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.94it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7963.36it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.98it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6076.06it/s]

 86%|████████▌ | 6/7 [02:04<00:15, 15.45s/it]

window_size: 1500
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.8633863758295774
MakeSCNNXExpDicUnknown Time:  8.588882570620626
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8806.01it/s]

 20%|██        | 1/5 [00:00<00:00,  5.07it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8931.65it/s]

 40%|████      | 2/5 [00:00<00:00,  5.10it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8180.82it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.98it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 17817.77it/s]

 80%|████████  | 4/5 [00:00<00:00,  5.02it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8691.06it/s]

100%|██████████| 7/7 [02:14<00:00, 19.27s/it]


In [5]:
key_column= test_dict_key#["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

NameError: name 'test_dict_key' is not defined

In [18]:
exp_deep_archi_extractors_dict.keys()

dict_keys(['SCNN_1_3_conv_1_dense_arg_dict_default', 'SCNN_3_123_conv_1_dense_arg_dict_default', 'SCNN_3_1_conv_1_dense_arg_dict_default', 'SCNN_1_2_conv_1_dense_arg_dict_default', 'SCNN_1_1_conv_1_dense_arg_dict_default', 'SCNN_1_2_conv_2_dense_arg_dict_default', 'SCNN_1_5_conv_1_dense_arg_dict_default', 'SCNN_4_1234_conv_1_dense_arg_dict_default'])

ERROR! Session/line number was not unique in database. History logging moved to new session 1227


In [19]:
K.clear_session()
tf.compat.v1.reset_default_graph()

In [ ]:
window_size_lst=WINDOW_SIZE_LST
exp_config=P
extract_features_func_dict=exp_deep_archi_extractors_dict["SCNN_1_3_conv_1_dense_arg_dict_default"]
dfList_exp1=dfList_dict["dfList_exp1"]
dfList_exp2=dfList_dict["dfList_exp2"]
EER_distro_df_dict = {}

if exp_config.classifier=="OCSVM":
    columns = ['window_size', "step_width", "Mean_EER", "median_nu", "median_gamma"]

elif exp_config.classifier=="IF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_estimators", "median_max_samples", "median_contamination", "median_max_features"]

elif exp_config.classifier=="LOF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_neighbors", "median_contamination"]

else:
    raise Exception("Unspecified classifier")

Mean_EER_df = pd.DataFrame(columns = columns)
for window_size in tqdm(window_size_lst):
    print(f"window_size: {window_size}")
    step_width = int(window_size * (1-overlap))


    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, test_set, exp_config.scaler, 
                                                                                     exp_config.num_sample_points_per_exp, EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                                     SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                                     Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                                     verbose=0)


    extract_features_func=extract_features_func_dict[window_size]

    X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict)
    
X_exp_test_dic


  0%|          | 0/2 [00:00<?, ?it/s]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10


  0%|          | 0/2 [00:02<?, ?it/s]


In [42]:
X_test_exp1_dict[0][0].shape

(1750, 6)

In [50]:
X_test_exp1_dict.keys()

dict_keys([0, 12, 14, 9, 18, 23, 2, 15, 10, 4])

In [38]:
#X_test_exp1_dict

In [28]:
base_path=f"final_experiments_results/{SAVE_PATH_NAME}/SCNN-OCSVM/{EXP_NUM_1_1}"
archi_name = list(TRAINING_CONFIG_DICT['2000'].keys())[0]
# archi_name="SCNN_3_1_conv_1_dense_arg_dict_default"
path_to_archi_name=base_path+f"/{archi_name}"
archi_win_dict={}
final_path=path_to_archi_name+f"/{1750}"
final_path

'final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1750'

In [35]:
loss_func = lambda y_true, dist: k_contrastive_loss(y_true, dist, arg_dict["contrastive_loss_margin"])

In [46]:
deep_feature_extractor_path=final_path+f"/SCNN-OCSVM-{EXP_NUM_1_1}-{archi_name}-{1750}-deep_feature_extractor"
print(deep_feature_extractor_path)

deep_feature_model=load_model(deep_feature_extractor_path)# custom_objects={'loss_func': loss_func})

final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.1/SCNN_1_3_conv_1_dense_arg_dict_default/1750/SCNN-OCSVM-1.1-SCNN_1_3_conv_1_dense_arg_dict_default-1750-deep_feature_extractor
ERROR! Session/line number was not unique in database. History logging moved to new session 1233


In [47]:
deep_feature_model.predict(np.array([X_test_exp1_dict[0][0]]))

array([[0.483646  , 0.48248574, 0.48391485, 0.5176617 , 0.48232555,
        0.3948905 , 0.6571675 , 0.48332578, 0.51537776, 0.3830801 ,
        0.48369867, 0.51706755, 0.6734431 , 0.48458332, 0.48752847,
        0.48411527, 0.40387475, 0.518302  , 0.5168323 , 0.48319304,
        0.5119249 , 0.36883786, 0.51656306, 0.48535657, 0.5160692 ,
        0.49645782, 0.6151099 , 0.3447324 , 0.5164715 , 0.5170238 ,
        0.48302537, 0.50929   , 0.48378727, 0.48427698, 0.37200788,
        0.4824282 , 0.5118454 , 0.5513184 , 0.5161465 , 0.52022976,
        0.51684374, 0.48289943, 0.5146176 , 0.3277162 , 0.5170986 ,
        0.51055324, 0.49970394, 0.48305914, 0.4838405 , 0.48391843,
        0.5158296 , 0.48297504, 0.48362806, 0.48273656, 0.5167648 ,
        0.4758899 , 0.5162947 , 0.48710927, 0.5125867 , 0.51675326,
        0.65629625, 0.5182427 , 0.6739007 , 0.665079  , 0.48348317,
        0.4852233 , 0.48333368, 0.3298231 , 0.48392308, 0.33389822,
        0.5162636 , 0.5160936 , 0.5179101 , 0.48

In [48]:
ExtractDeepFeatures([X_test_exp1_dict], deep_feature_model)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type dict).

In [ ]:
extract_learnt_features(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model, verbose=1)

In [ ]:
MakeDeepXExpDic(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, 
                    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model)


In [23]:
P.smoothing=None
preprocessing_method=None
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

,"('', 'scnn_archi_name')","('Window Size', 125)","('Window Size', 250)","('Window Size', 500)","('Window Size', 750)","('Window Size', 1000)","('Window Size', 1250)","('Window Size', 1500)","('', 'mean')","('', 'var')"
0,SCNN_3_1_conv_1_dense_arg_dict_default,0.121724,0.074870,0.068581,0.070040,0.052927,0.049861,0.064856,0.071837,0.000567
1,SCNN_4_1234_conv_1_dense_arg_dict_default,0.109947,0.092628,0.097711,0.111091,0.087886,0.123750,0.101975,0.103570,0.000151
2,SCNN_1_1_conv_1_dense_arg_dict_default,0.149548,0.107412,0.084270,0.140768,0.108564,0.072778,0.083621,0.106709,0.000864
3,SCNN_1_2_conv_2_dense_arg_dict_default,0.138351,0.124178,0.116921,0.088848,0.103089,0.098368,0.156749,0.118072,0.000569
4,SCNN_1_2_conv_1_dense_arg_dict_default,0.134573,0.126534,0.119826,0.112162,0.132737,0.104201,0.138601,0.124091,0.000160
5,SCNN_3_123_conv_1_dense_arg_dict_default,0.097349,0.106953,0.136894,0.142061,0.158347,0.169271,0.120947,0.133117,0.000692
6,SCNN_1_3_conv_1_dense_arg_dict_default,0.163574,0.089940,0.133173,0.197010,0.129051,0.167882,0.093457,0.139155,0.001570
7,SCNN_1_5_conv_1_dense_arg_dict_default,0.194774,0.147172,0.140669,0.158646,0.164011,0.205417,0.140988,0.164525,0.000674


In [24]:
P.smoothing="Butterworth"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

,"('', 'scnn_archi_name')","('', 'cut_off_freq')","('Window Size', 125)","('Window Size', 250)","('Window Size', 500)","('Window Size', 750)","('Window Size', 1000)","('Window Size', 1250)","('Window Size', 1500)","('', 'mean')","('', 'var')"
0,SCNN_3_1_conv_1_dense_arg_dict_default,41,0.115526,0.075156,0.060977,0.063475,0.051870,0.056042,0.061111,0.069165,0.000470
1,SCNN_1_1_conv_1_dense_arg_dict_default,41,0.158266,0.102522,0.084257,0.087455,0.073360,0.067604,0.096996,0.095780,0.000908
2,SCNN_1_2_conv_2_dense_arg_dict_default,41,0.146208,0.096261,0.098153,0.067131,0.114390,0.080833,0.135103,0.105440,0.000805
3,SCNN_1_2_conv_1_dense_arg_dict_default,41,0.141391,0.101490,0.112182,0.142485,0.128184,0.106875,0.136708,0.124188,0.000294
4,SCNN_4_1234_conv_1_dense_arg_dict_default,41,0.124316,0.106673,0.100455,0.111758,0.101843,0.142986,0.183086,0.124445,0.000891
5,SCNN_3_123_conv_1_dense_arg_dict_default,41,0.099172,0.081384,0.114592,0.159859,0.146938,0.169965,0.138724,0.130091,0.001067
6,SCNN_1_3_conv_1_dense_arg_dict_default,41,0.126281,0.095356,0.102798,0.209859,0.113415,0.152986,0.223210,0.146272,0.002662
7,SCNN_1_5_conv_1_dense_arg_dict_default,41,0.256373,0.172182,0.181392,0.122586,0.168808,0.154688,0.251481,0.186787,0.002458


In [25]:
P.smoothing="EMA"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

,"('', 'scnn_archi_name')","('', 'EMA_span')","('Window Size', 125)","('Window Size', 250)","('Window Size', 500)","('Window Size', 750)","('Window Size', 1000)","('', 'mean')","('', 'var')"
0,SCNN_3_1_conv_1_dense_arg_dict_default,25,0.108440,0.074518,0.079237,0.066788,0.061491,0.078095,0.000335
1,SCNN_1_2_conv_2_dense_arg_dict_default,25,0.125081,0.078470,0.076051,0.090727,0.075664,0.089199,0.000440
2,SCNN_3_123_conv_1_dense_arg_dict_default,25,0.110257,0.090326,0.098193,0.078000,0.110623,0.097480,0.000192
3,SCNN_1_2_conv_1_dense_arg_dict_default,25,0.131355,0.080326,0.110790,0.091475,0.100705,0.102930,0.000379
4,SCNN_4_1234_conv_1_dense_arg_dict_default,25,0.113406,0.080033,0.100870,0.124586,0.153875,0.114554,0.000757
5,SCNN_1_3_conv_1_dense_arg_dict_default,25,0.117188,0.087791,0.122718,0.119212,0.131951,0.115772,0.000277
6,SCNN_1_1_conv_1_dense_arg_dict_default,25,0.147468,0.116188,0.118701,0.103253,0.093930,0.115908,0.000412
7,SCNN_1_5_conv_1_dense_arg_dict_default,25,0.171576,0.253380,0.116064,0.123475,0.168509,0.166601,0.002993


In [26]:
P.smoothing="Butter+EMA"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

,"('', 'scnn_archi_name')","('', 'cut_off_freq')","('', 'EMA_span')","('Window Size', 125)","('Window Size', 250)","('Window Size', 500)","('Window Size', 750)","('Window Size', 1000)","('', 'mean')","('', 'var')"
0,SCNN_3_1_conv_1_dense_arg_dict_default,24,30,0.109044,0.075675,0.078942,0.070384,0.065447,0.079899,0.000292
1,SCNN_1_2_conv_2_dense_arg_dict_default,24,30,0.125971,0.094424,0.088809,0.104162,0.072981,0.097269,0.000385
2,SCNN_1_2_conv_1_dense_arg_dict_default,24,30,0.124425,0.079674,0.096881,0.105556,0.123306,0.105968,0.000354
3,SCNN_3_123_conv_1_dense_arg_dict_default,24,30,0.110455,0.093367,0.081071,0.129333,0.145312,0.111908,0.000679
4,SCNN_4_1234_conv_1_dense_arg_dict_default,24,30,0.122232,0.090872,0.106439,0.140040,0.100434,0.112003,0.000376
5,SCNN_1_3_conv_1_dense_arg_dict_default,24,30,0.131527,0.110213,0.107724,0.126788,0.138401,0.122931,0.000180
6,SCNN_1_1_conv_1_dense_arg_dict_default,24,30,0.136782,0.116241,0.104605,0.167939,0.105989,0.126311,0.000707
7,SCNN_1_5_conv_1_dense_arg_dict_default,24,30,0.181510,0.143180,0.163280,0.170303,0.163604,0.164376,0.000195


## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [27]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    test_dict_key=scnn_archi_name, P.cut_off_freq
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,  
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
#-------
#-------
key_column= ["scnn_archi_name", "cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-1.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.2/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-1.2-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.2/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-1.2-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.2/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-1.2-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/1.2/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-1.2-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.319145831745118
MakeSCNNXExpDicUnknown Time:  37.13779036235064
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9813.53it/s]

 20%|██        | 1/5 [00:07<00:29,  7.45s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10868.89it/s]

 40%|████      | 2/5 [00:08<00:10,  3.66s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8483.62it/s]

 60%|██████    | 3/5 [00:09<00:04,  2.42s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5626.16it/s]

 80%|████████  | 4/5 [00:10<00:01,  1.76s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 12246.14it/s]

 20%|██        | 1/5 [01:25<05:41, 85.28s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.169543051160872
MakeSCNNXExpDicUnknown Time:  21.830521434079856
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10710.68it/s]

 20%|██        | 1/5 [00:00<00:00,  4.18it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9586.98it/s]

 40%|████      | 2/5 [00:00<00:00,  4.28it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9615.55it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.11it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5515.19it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.09it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7949.78it/s]

 60%|██████    | 3/5 [02:28<01:26, 43.48s/it]

window_size: 750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.977533089928329
MakeSCNNXExpDicUnknown Time:  11.541805902030319
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6127.54it/s]

 20%|██        | 1/5 [00:00<00:00,  4.82it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 13119.50it/s]

 40%|████      | 2/5 [00:00<00:00,  4.74it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9025.83it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.70it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10353.75it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.70it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9455.15it/s]

 80%|████████  | 4/5 [02:48<00:34, 34.11s/it]

window_size: 1000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9692088798619807
MakeSCNNXExpDicUnknown Time:  10.279849838931113
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8538.89it/s]

 20%|██        | 1/5 [00:00<00:00,  4.90it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 12505.38it/s]

 40%|████      | 2/5 [00:00<00:00,  4.81it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8466.50it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.71it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8946.89it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.75it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10051.05it/s]

100%|██████████| 5/5 [03:05<00:00, 37.08s/it]


# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [20]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    


EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.cut_off_freq, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------  
#-------
key_column= ["scnn_archi_name", "cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/2.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-2.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/2.1/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-2.1-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/2.1/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-2.1-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/2.1/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-2.1-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/2.1/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-2.1-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10


2023-06-26 16:17:06.286457: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


MakeSCNNExpDicOwner Time:  1.5475563607178628
MakeSCNNXExpDicUnknown Time:  36.84844472585246
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8179.22it/s]

 20%|██        | 1/5 [00:00<00:00,  4.06it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 15779.92it/s]

 40%|████      | 2/5 [00:00<00:00,  4.01it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9191.99it/s]

 60%|██████    | 3/5 [00:00<00:00,  3.99it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7961.85it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.03it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9094.33it/s]

 60%|██████    | 3/5 [01:28<00:52, 26.28s/it]

window_size: 750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.979880781378597
MakeSCNNXExpDicUnknown Time:  11.669786059763283
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10128.72it/s]

 20%|██        | 1/5 [00:00<00:00,  4.64it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9395.84it/s]

 40%|████      | 2/5 [00:00<00:00,  4.58it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5492.08it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.62it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7153.85it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.62it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8692.86it/s]

 80%|████████  | 4/5 [01:42<00:21, 21.44s/it]

window_size: 1000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9945942331105471
MakeSCNNXExpDicUnknown Time:  10.55897887982428
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8689.26it/s]

 20%|██        | 1/5 [00:00<00:00,  4.61it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9745.13it/s]

 40%|████      | 2/5 [00:00<00:00,  4.80it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9523.85it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.99it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 12527.79it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.94it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 5919.14it/s]

100%|██████████| 5/5 [01:55<00:00, 23.07s/it]


## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")





P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    
    
    P.span=span
    P.EMA_per_win_span=span

    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [21]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    
        

    test_dict_key=scnn_archi_name, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-3.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.1/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-3.1-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.1/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-3.1-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.1/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-3.1-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.1/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-3.1-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.4046711320988834
MakeSCNNXExpDicUnknown Time:  37.088382449001074
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7986.11it/s]

 20%|██        | 1/5 [00:00<00:02,  2.00it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10104.32it/s]

 40%|████      | 2/5 [00:00<00:01,  2.04it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7751.44it/s]

 60%|██████    | 3/5 [00:01<00:00,  2.05it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8876.83it/s]

 80%|████████  | 4/5 [00:01<00:00,  2.01it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8393.64it/s]

 20%|██        | 1/5 [00:41<02:47, 41.93s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.1084894170053303
MakeSCNNXExpDicUnknown Time:  21.886202634312212
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8616.07it/s]

 20%|██        | 1/5 [00:00<00:01,  3.73it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7626.01it/s]

 40%|████      | 2/5 [00:00<00:00,  3.91it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6702.31it/s]

 60%|██████    | 3/5 [00:00<00:00,  3.97it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8977.53it/s]

 80%|████████  | 4/5 [00:01<00:00,  4.00it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8468.21it/s]

 60%|██████    | 3/5 [01:29<00:54, 27.13s/it]

window_size: 750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.0327439229004085
MakeSCNNXExpDicUnknown Time:  12.064299054909497
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8031.99it/s]

 20%|██        | 1/5 [00:00<00:00,  4.85it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 16156.80it/s]

 40%|████      | 2/5 [00:00<00:00,  4.64it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8053.58it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.60it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10338.44it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.70it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10710.68it/s]

 80%|████████  | 4/5 [01:44<00:22, 22.11s/it]

window_size: 1000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.036726108752191
MakeSCNNXExpDicUnknown Time:  11.005270359106362
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6396.68it/s]

 20%|██        | 1/5 [00:00<00:00,  4.98it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9960.35it/s]

 40%|████      | 2/5 [00:00<00:00,  5.02it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8090.86it/s]

 60%|██████    | 3/5 [00:00<00:00,  5.02it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10675.25it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.93it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 16100.98it/s]

100%|██████████| 5/5 [01:57<00:00, 23.53s/it]


## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [22]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    P.EMA_per_win_span=P.span
    print(f"EMA span: {P.span}")



    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    test_dict_key=scnn_archi_name, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict,  
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-3.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.2/SCNN_1_3_conv_1_dense_arg_dict_default/250/SCNN-OCSVM-3.2-SCNN_1_3_conv_1_dense_arg_dict_default-250-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.2/SCNN_1_3_conv_1_dense_arg_dict_default/500/SCNN-OCSVM-3.2-SCNN_1_3_conv_1_dense_arg_dict_default-500-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.2/SCNN_1_3_conv_1_dense_arg_dict_default/750/SCNN-OCSVM-3.2-SCNN_1_3_conv_1_dense_arg_dict_default-750-deep_feature_extractor
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/3.2/SCNN_1_3_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-3.2-SCNN_1_3_conv_1_dense_arg_dict_default-1000-deep_feature_extractor
final_experiments_results/SCNN_tra

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.4215360642410815
MakeSCNNXExpDicUnknown Time:  38.30208279378712
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9556.40it/s]

 20%|██        | 1/5 [00:00<00:01,  2.04it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7397.36it/s]

 40%|████      | 2/5 [00:00<00:01,  2.03it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7212.90it/s]

 60%|██████    | 3/5 [00:01<00:00,  2.03it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8276.05it/s]

 80%|████████  | 4/5 [00:01<00:00,  2.12it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8556.31it/s]

 20%|██        | 1/5 [00:49<03:18, 49.56s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.1486545065417886
MakeSCNNXExpDicUnknown Time:  22.364152270834893
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10145.87it/s]

 20%|██        | 1/5 [00:00<00:00,  4.16it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7518.02it/s]

 40%|████      | 2/5 [00:00<00:00,  4.20it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8912.67it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.20it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9252.82it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.15it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10145.87it/s]

 60%|██████    | 3/5 [01:45<01:02, 31.08s/it]

window_size: 750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  1.0722034657374024
MakeSCNNXExpDicUnknown Time:  12.421656815800816
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 5954.43it/s]

 20%|██        | 1/5 [00:00<00:00,  4.67it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7966.39it/s]

 40%|████      | 2/5 [00:00<00:00,  4.67it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5878.49it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.60it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7909.30it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.55it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8653.40it/s]

 80%|████████  | 4/5 [02:01<00:25, 25.13s/it]

window_size: 1000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeSCNNExpDicOwner Time:  0.9977388917468488
MakeSCNNXExpDicUnknown Time:  11.25878551416099
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7747.14it/s]

 20%|██        | 1/5 [00:00<00:00,  4.75it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9955.62it/s]

 40%|████      | 2/5 [00:00<00:00,  4.80it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10968.37it/s]

 60%|██████    | 3/5 [00:00<00:00,  4.83it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 11466.11it/s]

 80%|████████  | 4/5 [00:00<00:00,  4.81it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 14261.49it/s]

100%|██████████| 5/5 [02:15<00:00, 27.10s/it]


# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [58]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.winsize
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/4.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-4.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor


OSError: SavedModel file does not exist at: final_experiments_results/SCNN_trained_models/SCNN-OCSVM/4.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-4.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor/{saved_model.pbtxt|saved_model.pb}

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [59]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]    
    P.SMA_per_win_winsize=P.winsize
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.winsize
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/4.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-4.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor


OSError: SavedModel file does not exist at: final_experiments_results/SCNN_trained_models/SCNN-OCSVM/4.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-4.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor/{saved_model.pbtxt|saved_model.pb}

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [79]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.winsize=winsize
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/5.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-5.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor


OSError: SavedModel file does not exist at: final_experiments_results/SCNN_trained_models/SCNN-OCSVM/5.1/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-5.1-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor/{saved_model.pbtxt|saved_model.pb}

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [78]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")




P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize
    
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
final_experiments_results/SCNN_trained_models/SCNN-OCSVM/5.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-5.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor


OSError: SavedModel file does not exist at: final_experiments_results/SCNN_trained_models/SCNN-OCSVM/5.2/SCNN_1_3_conv_1_dense_arg_dict_default/125/SCNN-OCSVM-5.2-SCNN_1_3_conv_1_dense_arg_dict_default-125-deep_feature_extractor/{saved_model.pbtxt|saved_model.pb}

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    


EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(list(range(50, 52))):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=min(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 20)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [ ]:
1

# Summary

In [ ]:

df=return_and_save_final_result_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
df

In [ ]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, 
                                                             window_size_lst=WINDOW_SIZE_LST)
relative_df

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
